In [1]:
import category_encoders as ce
import numpy as np
import pandas as pd
from xgboost import XGBRegressor

history = pd.read_csv('lending-club.csv')
X = history.drop(columns='Interest Rate')
y = history['Interest Rate']
y_log = np.log1p(y)

encoder = ce.OneHotEncoder(use_cat_names=True)
model = XGBRegressor(n_estimators=200, n_jobs=-1, random_state=42)

X_encoded = encoder.fit_transform(X)
model.fit(X_encoded.values, y_log)

/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[02:06:05] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=200,
             n_jobs=-1, nthread=None, objective='reg:linear', random_state=42,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [2]:
%%time
from sklearn.inspection import partial_dependence
features = [X_encoded.columns.get_loc('Annual Income'), 
            X_encoded.columns.get_loc('Credit Score')]
pdp, axes = partial_dependence(model, X_encoded, features=features)

CPU times: user 19min 24s, sys: 6.85 s, total: 19min 30s
Wall time: 19min 32s


In [91]:
from joblib import dump, load

dump(axes[0], 'x.joblib')
dump(axes[1], 'y.joblib')
dump(np.expm1(pdp[0]), 'z.joblib')

x = load('x.joblib')
y = load('y.joblib')
z = load('z.joblib')

In [92]:
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

In [93]:
import plotly.graph_objs as go

In [95]:
data = [
    go.Surface(
        z=z, 
        x=x, 
        y=y
    )
]
layout = go.Layout(
    title='Partial Dependence Plot',
    scene={'xaxis': {'title': 'Annual Income', 'showticklabels': True}, 
           'yaxis': {'title': 'Credit Score', 'showticklabels': True}, 
           'zaxis': {'title': 'Interest Rate', 'showticklabels': True}}, 
)
fig = go.Figure(data=data, layout=layout)
iplot(fig)